In [42]:
#Load data into dataframe
data = spark.read.csv("file:/home/hduser/hive/data/txns", inferSchema = True)\
.toDF("txnid","txndate","custid","amount","category","product","city","state","paymenttype")
data.show()


+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-----------+
|txnid|   txndate| custid|amount|            category|             product|          city|         state|paymenttype|
+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-----------+
|    0|06-26-2011|4007024| 40.33|  Exercise & Fitness|Cardio Machine Ac...|   Clarksville|     Tennessee|     credit|
|    1|05-26-2011|4006742|198.44|  Exercise & Fitness|Weightlifting Gloves|    Long Beach|    California|     credit|
|    2|06-01-2011|4009775|  5.58|  Exercise & Fitness|Weightlifting Mac...|       Anaheim|    California|     credit|
|    3|06-05-2011|4002199|198.19|          Gymnastics|    Gymnastics Rings|     Milwaukee|     Wisconsin|     credit|
|    4|12-17-2011|4002613| 98.81|         Team Sports|        Field Hockey|   Nashville  |     Tennessee|     credit|
|    5|02-14-2011|4007591|193.63|  Outdoor Recreation|Ca

In [43]:
from pyspark.sql.functions import col, unix_timestamp,year,month,dayofmonth
data = data.withColumn("sales_date", unix_timestamp(col("txndate"), "MM-dd-yyyy").cast("timestamp"))
data = data.withColumn("year", year(col("sales_date")))
data = data.withColumn("month", month(col("sales_date")))
data = data.withColumn("day", dayofmonth(col("sales_date")))
data.printSchema()
data.show()

root
 |-- txnid: integer (nullable = true)
 |-- txndate: string (nullable = true)
 |-- custid: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- paymenttype: string (nullable = true)
 |-- sales_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)

+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-----------+-------------------+----+-----+---+
|txnid|   txndate| custid|amount|            category|             product|          city|         state|paymenttype|         sales_date|year|month|day|
+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-----------+-------------------+----+-----+---+
|    0|06-26-2011|4007024| 40.

In [44]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
indexers = [
    StringIndexer(inputCol="product", outputCol="product_index"),
    StringIndexer(inputCol="category", outputCol="category_index"),
    StringIndexer(inputCol="city", outputCol="city_index"),
    StringIndexer(inputCol="state", outputCol="state_index")
]
for indexer in indexers:
    data = indexer.fit(data).transform(data)

data.printSchema()
data.show()

root
 |-- txnid: integer (nullable = true)
 |-- txndate: string (nullable = true)
 |-- custid: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- paymenttype: string (nullable = true)
 |-- sales_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- product_index: double (nullable = false)
 |-- category_index: double (nullable = false)
 |-- city_index: double (nullable = false)
 |-- state_index: double (nullable = false)

+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-----------+-------------------+----+-----+---+-------------+--------------+----------+-----------+
|txnid|   txndate| custid|amount|            category|             product|          city|         state

In [45]:
assembler = VectorAssembler(
    inputCols=["product_index", "category_index", "city_index", "state_index", "year","month","day"],
    outputCol="features"
)
data = assembler.transform(data)

data.printSchema()

root
 |-- txnid: integer (nullable = true)
 |-- txndate: string (nullable = true)
 |-- custid: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- paymenttype: string (nullable = true)
 |-- sales_date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- product_index: double (nullable = false)
 |-- category_index: double (nullable = false)
 |-- city_index: double (nullable = false)
 |-- state_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [46]:
final_data = data.select(col("features"), col("amount").alias("label"))

#Split Data
train_data, test_data = final_data.randomSplit([0.8, 0.2])


In [47]:
#Train Random Forest Model
from pyspark.ml.regression import RandomForestRegressor
rt_rf = RandomForestRegressor(featuresCol="features", labelCol="label", maxBins=320)
rf_model = rt_rf.fit(train_data)
rt_predictions = rf_model.transform(test_data)
rt_predictions.select("prediction", "label", "features").show()

+------------------+------+--------------------+
|        prediction| label|            features|
+------------------+------+--------------------+
| 108.3122928753364| 41.88|[0.0,1.0,0.0,11.0...|
|109.62246119909537|192.62|[0.0,1.0,0.0,19.0...|
|106.49988137274354|184.29|[0.0,1.0,1.0,11.0...|
|106.18573273069539|112.39|[0.0,1.0,3.0,0.0,...|
|106.08837685066294| 44.76|[0.0,1.0,3.0,0.0,...|
|106.08837685066294| 98.09|[0.0,1.0,3.0,0.0,...|
|108.40236005411639|167.28|[0.0,1.0,4.0,16.0...|
|108.40236005411639|146.31|[0.0,1.0,4.0,16.0...|
|106.17813289524972| 95.48|[0.0,1.0,4.0,16.0...|
| 108.8220283132974|147.16|[0.0,1.0,4.0,16.0...|
|105.66608615060227| 67.24|[0.0,1.0,5.0,3.0,...|
| 104.8242201312546|167.47|[0.0,1.0,6.0,5.0,...|
| 104.8242201312546|  36.4|[0.0,1.0,6.0,5.0,...|
|105.34357512527599| 61.57|[0.0,1.0,6.0,5.0,...|
|101.67127821332079| 98.37|[0.0,1.0,8.0,3.0,...|
|109.37221263663227| 43.22|[0.0,1.0,9.0,4.0,...|
|110.35069560152719| 34.16|[0.0,1.0,9.0,4.0,...|
|110.36661903061106|

In [48]:
from pyspark.ml.regression import DecisionTreeRegressor
rt_dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", maxBins=320)
dt_model = rt_dt.fit(train_data)
dt_predictions = dt_model.transform(test_data)
dt_predictions.select("prediction", "label", "features").show()



+------------------+------+--------------------+
|        prediction| label|            features|
+------------------+------+--------------------+
|114.58040969417198| 41.88|[0.0,1.0,0.0,11.0...|
|114.58040969417198|192.62|[0.0,1.0,0.0,19.0...|
|101.99026475037822|184.29|[0.0,1.0,1.0,11.0...|
|101.99026475037822|112.39|[0.0,1.0,3.0,0.0,...|
|101.99026475037822| 44.76|[0.0,1.0,3.0,0.0,...|
|101.99026475037822| 98.09|[0.0,1.0,3.0,0.0,...|
|109.82799136069113|167.28|[0.0,1.0,4.0,16.0...|
|109.82799136069113|146.31|[0.0,1.0,4.0,16.0...|
|109.82799136069113| 95.48|[0.0,1.0,4.0,16.0...|
|109.82799136069113|147.16|[0.0,1.0,4.0,16.0...|
| 97.56988999476167| 67.24|[0.0,1.0,5.0,3.0,...|
| 97.56988999476167|167.47|[0.0,1.0,6.0,5.0,...|
| 97.56988999476167|  36.4|[0.0,1.0,6.0,5.0,...|
| 97.56988999476167| 61.57|[0.0,1.0,6.0,5.0,...|
| 97.56988999476167| 98.37|[0.0,1.0,8.0,3.0,...|
|101.99026475037822| 43.22|[0.0,1.0,9.0,4.0,...|
|101.99026475037822| 34.16|[0.0,1.0,9.0,4.0,...|
|101.99026475037822|

In [49]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(rt_predictions)
print(f"Root Mean Squared Error (RMSE) on test data for RandomForest = {rmse}")

rmse = evaluator.evaluate(dt_predictions)
print(f"Root Mean Squared Error (RMSE) on test data for DecisionTree = {rmse}")

Root Mean Squared Error (RMSE) on test data for RandomForest = 56.43196601537446
Root Mean Squared Error (RMSE) on test data for DecisionTree = 56.54876232593201
